In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup

# = = = = = = = = = = = = = = =

import re
import json

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'LXnMWcYQhipLAS7rImEzmZ3CkrU033FMha9cwVSngG4vbufTsAOCQQ=='

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.product_url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['[Product] No'] = ['' if product_no == '' else int(product_no) for product_no in df_input['[Product] No'].tolist()]

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：1



,Url,[Product] No,[Product] Url,Unnamed: 3
0,https://automechanika-shanghai.hk.messefrankfu...,2,https://automechanika-shanghai.hk.messefrankfu...,


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Url              https://automechanika-shanghai.hk.messefrankfu...
 [Product] No                                                     2
 [Product] Url    https://automechanika-shanghai.hk.messefrankfu...
 Unnamed: 3                                                        
 Name: 0, dtype: object)

In [5]:
crawler_series['[Product] Url'] = '' if not crawler_series['[Product] Url'] else crawler_series['[Product] Url']

crawler_series['[Product] Url']

'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/yongkang-mijia-imp--exp-coltd/mf_2_a4wc0a03divo_4061554_10000100202401.html'

In [6]:
crawler_series['[Product] Url'] == ''

False

In [7]:
count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request, url_check = f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/product/zh-CN/{crawler_series['[Product] Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI/details''', crawler_series['[Product] Url']

        resp = requests.get(url_request,
                            headers=get_header(**{'Apikey': api}),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
            print(f'fuck api {api}')
            time.sleep(3)
            with open('./api.txt', 'r') as api_txt:
                api = api_txt.read()
        elif resp.status_code == 200:
            dict_ = resp.json()

            if dict_['success'] == True:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1
2
3
4
5
6
7
8
9
10


{'success': False,
 'fieldErrors': [],
 'systemErrors': [{'system': 'FIND',
   'statusCode': 400,
   'description': 'FIND error: null',
   'url': 'https://api.main.messefrankfurt.com/service/find/mpm-api/v3/query/product/zh-CN/yongkang-mijia-imp--exp-coltd/mf_2_a4wc0a03divo_4061554_10000100202401/AUTOMECHANIKASHANGHAI/details'}],
 'message': None,
 'result': None}

In [8]:
dict_src, list_src = {}, dict_['result']['description']['images']
for i, src in enumerate(list_src):
    dict_src[str(i)] = f'''https://exhibitorsearch.hk.messefrankfurt.com/images/large/{src.strip()}'''

dict_src

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1083e3f80>>
Traceback (most recent call last):
  File "/Users/hzm/Documents/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


TypeError: 'NoneType' object is not subscriptable

In [ ]:
if dict_['result']['description']['text']:
    soup = BeautifulSoup(dict_['result']['description']['text'], 'lxml')
    description = re.sub(r'\t+', '\n', soup.get_text()).strip()
else:
    description = ''

print(description)

In [ ]:
list_dict_correct.append({'Url': crawler_series['Url'],
                          '[Product] No': crawler_series['[Product] No'],
                          '[Product] Name': dict_['result']['name'].strip(),
                          '[Product] Description': description,
                          '[Product] Picture': '',
                          '[Product] Url': crawler_series['[Product] Url'],
                          '[Product] Json_Src': json.dumps(dict_src)})

list_dict_correct

In [ ]:
crawler_status = True

crawler_status

In [ ]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

In [ ]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_product.xlsx', index=False)

df_correct

In [ ]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Url', '[Product] No'],
                                                           ascending=[True, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_product_error.xlsx', index=False)

df_error